In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

from datetime import datetime, timedelta, timezone

import sys
sys.path.append('../')
from utils import data
from models.lenet import LenetKeras as Lenet
from utils.visualization import *

In [2]:
bn = True
# const
taskName = 'fasionClassfiction'
modelName = 'lenetKeras{}'.format('_bn' if bn else '')
JST = timezone(timedelta(hours=+9), 'JST')
now = datetime.now(JST)
nowStr = now.strftime("%Y%m%d_%H%M%S")
showImgCount = 10

imgHeight = 224
imgWidth = 224
imgChannel = 3
imgShape = [-1, imgHeight, imgWidth, imgChannel]
labelSize = 13
tfrecordPath = './img/fashionDataset/tfrecord/dataset224.tfrecord'
trainRatio = 0.7

# hyper parameter
bs = 32
lr = 0.00001
ep = 50

logDir = '../logs/{}/{}/{}/'.format(taskName, modelName, nowStr)
metadataDir = '{}metadata.tsv'.format(logDir)
checkPointDir = '{}images.ckpt'.format(logDir)

In [3]:
# main
with tf.Graph().as_default():
    # load data
    with tf.variable_scope('tfrecord'):
        tfrecord = data.TFRecord(tfrecordPath, labelSize)
        dataset = tfrecord.toDataset()
        # split dataset
        tfrecord = data.TFRecord(tfrecordPath, labelSize)
        dataset = tfrecord.toDataset()
        trainSize, testSize, trainDataset, testDataset = tfrecord.splitDataset(bs, trainRatio)
        trainIteration = trainSize // bs
        testIteration = testSize // bs        
        # TODO: data augmentation

    # make iterator
    with tf.variable_scope('train_data'):
        trainIterator = trainDataset.make_initializable_iterator()
        trainNextBatch = trainIterator.get_next(name='train_next_batch')
        trainIteratorInitOp = trainIterator.initializer
    with tf.variable_scope('test_data'):
        testIterator = testDataset.make_initializable_iterator()
        testNextBatch = testIterator.get_next(name='test_next_batch')
        testIteratorInitOp = testIterator.initializer

    
    x = tf.placeholder("float", [None, imgHeight, imgWidth, imgChannel], name='x')
    y = tf.placeholder("float", [None, labelSize], name='label')
    isTrain = tf.placeholder(tf.bool, name='is_train')
    
    network = Lenet(x, inputReshapeTo=imgShape, labelSize=labelSize, batchNorm=bn)
    _ = network.inference(isTrain)
    cost = network.loss(y)
    optimizer = tf.train.AdamOptimizer(lr)
    accuracy = network.accuracy(y)
    trainOp = network.training(cost, optimizer)
    
    # tensorBoard
    allVariables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
    drawOp = []
    for v in allVariables:
        op = tf.summary.histogram(v.name, v)
        drawOp.append(op)
    summaryOp = tf.summary.merge([drawOp])
    
    saver = tf.train.Saver()
    config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
    sess = tf.Session(config=config)
    summaryWriter = tf.summary.FileWriter(logDir, graph=sess.graph)
    sess.run(tf.global_variables_initializer())
    
    # Training cycle
    for e in range(ep):
        trainAvgAcc = trainAvgLoss = testAvgAcc = testAvgLoss = 0.
        # train
        sess.run(trainIteratorInitOp)
        for i in range(trainIteration):
            _, batchX, batchY = sess.run(trainNextBatch)
            _, l, a = sess.run([trainOp, cost, accuracy], feed_dict={x: batchX, y: batchY, isTrain: True})
            trainAvgAcc += a / trainIteration
            trainAvgLoss += l / trainIteration

        # validation
        sess.run(testIteratorInitOp)
        for i in range(testIteration):
            _, testBatchX, testBatchY = sess.run(testNextBatch)
            l, a, summary = sess.run([cost, accuracy, summaryOp], feed_dict={x: testBatchX, y: testBatchY, isTrain: False})
            testAvgAcc += a / testIteration
            testAvgLoss += l / testIteration

        # tensorBoard
        summaryWriter.add_summary(drawSclar('train', {'acc': trainAvgAcc, 'loss': trainAvgLoss}), global_step=e)
        summaryWriter.add_summary(drawSclar('validation', {'acc': testAvgAcc, 'loss': testAvgLoss}), global_step=e)
        summaryWriter.add_summary(summary, global_step=e)
        saver.save(sess, checkPointDir, global_step=e)
        print('Epoch: {} | Train Accuracy: {:.4f} | Train Loss: {:.4f} | Test Accuracy: {:.4f} | Test Loss: {:.4f}'.format(e, trainAvgAcc, trainAvgLoss, testAvgAcc, testAvgLoss))
    summaryWriter.close()
    sess.close()

INFO:tensorflow:Summary name conv_1/conv2d/kernel:0 is illegal; using conv_1/conv2d/kernel_0 instead.
INFO:tensorflow:Summary name conv_1/conv2d/bias:0 is illegal; using conv_1/conv2d/bias_0 instead.
INFO:tensorflow:Summary name conv_1/batch_normalization/gamma:0 is illegal; using conv_1/batch_normalization/gamma_0 instead.
INFO:tensorflow:Summary name conv_1/batch_normalization/beta:0 is illegal; using conv_1/batch_normalization/beta_0 instead.
INFO:tensorflow:Summary name conv_1/batch_normalization/moving_mean:0 is illegal; using conv_1/batch_normalization/moving_mean_0 instead.
INFO:tensorflow:Summary name conv_1/batch_normalization/moving_variance:0 is illegal; using conv_1/batch_normalization/moving_variance_0 instead.
INFO:tensorflow:Summary name conv_2/conv2d_1/kernel:0 is illegal; using conv_2/conv2d_1/kernel_0 instead.
INFO:tensorflow:Summary name conv_2/conv2d_1/bias:0 is illegal; using conv_2/conv2d_1/bias_0 instead.
INFO:tensorflow:Summary name conv_2/batch_normalization_1/

Epoch: 33 | Train Accuracy: 0.8520 | Train Loss: 1.8403 | Test Accuracy: 0.8496 | Test Loss: 1.8419
Epoch: 34 | Train Accuracy: 0.8549 | Train Loss: 1.8372 | Test Accuracy: 0.8543 | Test Loss: 1.8372
Epoch: 35 | Train Accuracy: 0.8564 | Train Loss: 1.8353 | Test Accuracy: 0.8543 | Test Loss: 1.8369
Epoch: 36 | Train Accuracy: 0.8565 | Train Loss: 1.8347 | Test Accuracy: 0.8572 | Test Loss: 1.8345
Epoch: 37 | Train Accuracy: 0.8567 | Train Loss: 1.8345 | Test Accuracy: 0.8538 | Test Loss: 1.8381
Epoch: 38 | Train Accuracy: 0.8553 | Train Loss: 1.8356 | Test Accuracy: 0.8538 | Test Loss: 1.8364
Epoch: 39 | Train Accuracy: 0.8578 | Train Loss: 1.8331 | Test Accuracy: 0.8560 | Test Loss: 1.8353
Epoch: 40 | Train Accuracy: 0.8595 | Train Loss: 1.8312 | Test Accuracy: 0.8571 | Test Loss: 1.8328
Epoch: 41 | Train Accuracy: 0.8553 | Train Loss: 1.8360 | Test Accuracy: 0.8559 | Test Loss: 1.8351
Epoch: 42 | Train Accuracy: 0.8597 | Train Loss: 1.8316 | Test Accuracy: 0.8549 | Test Loss: 1.8348
